In [1]:
#sklearn
from sklearn.cross_validation import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error,confusion_matrix, precision_score, recall_score, auc,roc_curve
from sklearn import ensemble, linear_model
from sklearn.metrics import log_loss


#load package
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#from math import sqrt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', 500)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [30]:
# read data
train_df_m=pd.read_csv('dinesh/competitions/crowdanalytix/tennis/mens_train_file.csv',index_col='id')
train_df_f=pd.read_csv('dinesh/competitions/crowdanalytix/tennis/womens_train_file.csv',index_col='id')
test_df_m=pd.read_csv('dinesh/competitions/crowdanalytix/tennis/mens_test_file.csv',index_col='id')
test_df_w=pd.read_csv('dinesh/competitions/crowdanalytix/tennis/womens_test_file.csv',index_col='id')

#remove target variable
men_outcome=train_df_m['outcome'].copy()
train_df_m=train_df_m.drop('outcome',axis=1).copy()
test_df_m=test_df_m.drop('outcome',axis=1).copy()



# join both train and test dataset to prepare data
df_m=pd.concat([train_df_m,test_df_m])
men_traindex=train_df_m.index
men_testdex=test_df_m.index
del train_df_m
del test_df_m

In [31]:
#create variables
df_m['pes.above.net']=np.where(df_m['net.clearance']>0,1,0)
df_m.head()

,rally,serve,hitpoint,speed,net.clearance,distance.from.sideline,depth,outside.sideline,outside.baseline,player.distance.travelled,player.impact.depth,player.impact.distance.from.center,player.depth,player.distance.from.center,previous.speed,previous.net.clearance,previous.distance.from.sideline,previous.depth,opponent.depth,opponent.distance.from.center,same.side,previous.hitpoint,previous.time.to.net,server.is.impact.player,train,gender,pes.above.net
id,,,,,,,,,,,,,,,,,,,,,,,,,,,
5718,4,1,B,35.515042,-0.021725,3.474766,6.797621,False,False,1.467570,14.1758,1.1448,13.6704,0.2330,32.313592,1.350533,2.449182,0.705435,12.5628,2.0724,True,F,0.445318,False,1,mens,0
371,4,2,B,33.382640,1.114202,2.540801,2.608708,False,True,2.311931,12.7396,4.6258,12.6358,2.3162,34.060788,0.760499,0.583291,3.856600,12.3544,5.1124,False,B,0.432434,False,1,mens,1
7948,23,1,B,22.316690,-0.254046,3.533166,9.435749,False,False,3.903728,15.3782,1.4336,13.6996,2.0908,39.806628,0.377732,1.115250,2.908892,13.8620,1.6564,False,F,0.397538,True,1,mens,0
9312,9,1,F,36.837309,0.766694,0.586885,3.342180,True,False,0.583745,13.7308,0.0312,13.1492,0.0188,25.018818,1.239033,3.256695,0.557554,14.2596,0.1606,True,B,0.671984,True,1,mens,1
7667,4,1,B,35.544208,0.116162,0.918725,5.499119,False,False,2.333456,15.1992,1.7596,14.1310,0.3150,36.790979,0.165731,1.431146,3.945317,11.3658,1.1082,False,F,0.340411,False,1,mens,1


In [32]:
bins=[-20,-0.0000000000001,0,3.5,20]
labels=['back','still','fwd','very fwd']
df_m['player.movement']=pd.cut((df_m['player.depth']-df_m['player.impact.depth']), bins=bins, labels=labels)
df_m.head()

,rally,serve,hitpoint,speed,net.clearance,distance.from.sideline,depth,outside.sideline,outside.baseline,player.distance.travelled,player.impact.depth,player.impact.distance.from.center,player.depth,player.distance.from.center,previous.speed,previous.net.clearance,previous.distance.from.sideline,previous.depth,opponent.depth,opponent.distance.from.center,same.side,previous.hitpoint,previous.time.to.net,server.is.impact.player,train,gender,pes.above.net,player.movement
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5718,4,1,B,35.515042,-0.021725,3.474766,6.797621,False,False,1.467570,14.1758,1.1448,13.6704,0.2330,32.313592,1.350533,2.449182,0.705435,12.5628,2.0724,True,F,0.445318,False,1,mens,0,back
371,4,2,B,33.382640,1.114202,2.540801,2.608708,False,True,2.311931,12.7396,4.6258,12.6358,2.3162,34.060788,0.760499,0.583291,3.856600,12.3544,5.1124,False,B,0.432434,False,1,mens,1,back
7948,23,1,B,22.316690,-0.254046,3.533166,9.435749,False,False,3.903728,15.3782,1.4336,13.6996,2.0908,39.806628,0.377732,1.115250,2.908892,13.8620,1.6564,False,F,0.397538,True,1,mens,0,back
9312,9,1,F,36.837309,0.766694,0.586885,3.342180,True,False,0.583745,13.7308,0.0312,13.1492,0.0188,25.018818,1.239033,3.256695,0.557554,14.2596,0.1606,True,B,0.671984,True,1,mens,1,back
7667,4,1,B,35.544208,0.116162,0.918725,5.499119,False,False,2.333456,15.1992,1.7596,14.1310,0.3150,36.790979,0.165731,1.431146,3.945317,11.3658,1.1082,False,F,0.340411,False,1,mens,1,back


In [33]:
bins1=[0,6.5,11.9,25]
labels1=['inside.sl','sl.to.bl','outside.bl']
df_m['player.penul.pos']=pd.cut((df_m['player.depth']), bins=bins1, labels=labels1)
df_m['player.impact.pos']=pd.cut((df_m['player.impact.depth']), bins=bins1, labels=labels1)
df_m['opp.penul.pos']=pd.cut((df_m['opponent.depth']), bins=bins1, labels=labels1)
df_m.head()

,rally,serve,hitpoint,speed,net.clearance,distance.from.sideline,depth,outside.sideline,outside.baseline,player.distance.travelled,player.impact.depth,player.impact.distance.from.center,player.depth,player.distance.from.center,previous.speed,previous.net.clearance,previous.distance.from.sideline,previous.depth,opponent.depth,opponent.distance.from.center,same.side,previous.hitpoint,previous.time.to.net,server.is.impact.player,train,gender,pes.above.net,player.movement,player.penul.pos,player.impact.pos,opp.penul.pos
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5718,4,1,B,35.515042,-0.021725,3.474766,6.797621,False,False,1.467570,14.1758,1.1448,13.6704,0.2330,32.313592,1.350533,2.449182,0.705435,12.5628,2.0724,True,F,0.445318,False,1,mens,0,back,outside.bl,outside.bl,outside.bl
371,4,2,B,33.382640,1.114202,2.540801,2.608708,False,True,2.311931,12.7396,4.6258,12.6358,2.3162,34.060788,0.760499,0.583291,3.856600,12.3544,5.1124,False,B,0.432434,False,1,mens,1,back,outside.bl,outside.bl,outside.bl
7948,23,1,B,22.316690,-0.254046,3.533166,9.435749,False,False,3.903728,15.3782,1.4336,13.6996,2.0908,39.806628,0.377732,1.115250,2.908892,13.8620,1.6564,False,F,0.397538,True,1,mens,0,back,outside.bl,outside.bl,outside.bl
9312,9,1,F,36.837309,0.766694,0.586885,3.342180,True,False,0.583745,13.7308,0.0312,13.1492,0.0188,25.018818,1.239033,3.256695,0.557554,14.2596,0.1606,True,B,0.671984,True,1,mens,1,back,outside.bl,outside.bl,outside.bl
7667,4,1,B,35.544208,0.116162,0.918725,5.499119,False,False,2.333456,15.1992,1.7596,14.1310,0.3150,36.790979,0.165731,1.431146,3.945317,11.3658,1.1082,False,F,0.340411,False,1,mens,1,back,outside.bl,outside.bl,sl.to.bl


In [34]:
bins2=[0,5.5,11.9]
labels2=['sl.to.bl','inside.sl']
df_m['penul.ball.bounce']=pd.cut((df_m['previous.depth']), bins=bins2, labels=labels2)

In [35]:
df_m.head()

,rally,serve,hitpoint,speed,net.clearance,distance.from.sideline,depth,outside.sideline,outside.baseline,player.distance.travelled,player.impact.depth,player.impact.distance.from.center,player.depth,player.distance.from.center,previous.speed,previous.net.clearance,previous.distance.from.sideline,previous.depth,opponent.depth,opponent.distance.from.center,same.side,previous.hitpoint,previous.time.to.net,server.is.impact.player,train,gender,pes.above.net,player.movement,player.penul.pos,player.impact.pos,opp.penul.pos,penul.ball.bounce
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5718,4,1,B,35.515042,-0.021725,3.474766,6.797621,False,False,1.467570,14.1758,1.1448,13.6704,0.2330,32.313592,1.350533,2.449182,0.705435,12.5628,2.0724,True,F,0.445318,False,1,mens,0,back,outside.bl,outside.bl,outside.bl,sl.to.bl
371,4,2,B,33.382640,1.114202,2.540801,2.608708,False,True,2.311931,12.7396,4.6258,12.6358,2.3162,34.060788,0.760499,0.583291,3.856600,12.3544,5.1124,False,B,0.432434,False,1,mens,1,back,outside.bl,outside.bl,outside.bl,sl.to.bl
7948,23,1,B,22.316690,-0.254046,3.533166,9.435749,False,False,3.903728,15.3782,1.4336,13.6996,2.0908,39.806628,0.377732,1.115250,2.908892,13.8620,1.6564,False,F,0.397538,True,1,mens,0,back,outside.bl,outside.bl,outside.bl,sl.to.bl
9312,9,1,F,36.837309,0.766694,0.586885,3.342180,True,False,0.583745,13.7308,0.0312,13.1492,0.0188,25.018818,1.239033,3.256695,0.557554,14.2596,0.1606,True,B,0.671984,True,1,mens,1,back,outside.bl,outside.bl,outside.bl,sl.to.bl
7667,4,1,B,35.544208,0.116162,0.918725,5.499119,False,False,2.333456,15.1992,1.7596,14.1310,0.3150,36.790979,0.165731,1.431146,3.945317,11.3658,1.1082,False,F,0.340411,False,1,mens,1,back,outside.bl,outside.bl,sl.to.bl,sl.to.bl


In [36]:
df_m = pd.get_dummies(df_m, columns=['serve','hitpoint','outside.sideline','outside.baseline','same.side','previous.hitpoint','server.is.impact.player','pes.above.net','player.movement','player.penul.pos','player.impact.pos','opp.penul.pos','penul.ball.bounce'],drop_first=True )
df_m.head()

,rally,speed,net.clearance,distance.from.sideline,depth,player.distance.travelled,player.impact.depth,player.impact.distance.from.center,player.depth,player.distance.from.center,previous.speed,previous.net.clearance,previous.distance.from.sideline,previous.depth,opponent.depth,opponent.distance.from.center,previous.time.to.net,train,gender,serve_2,hitpoint_F,hitpoint_U,hitpoint_V,outside.sideline_True,outside.baseline_True,same.side_True,previous.hitpoint_F,previous.hitpoint_U,previous.hitpoint_V,server.is.impact.player_True,pes.above.net_1,player.movement_fwd,player.movement_still,player.movement_very fwd,player.penul.pos_outside.bl,player.penul.pos_sl.to.bl,player.impact.pos_outside.bl,player.impact.pos_sl.to.bl,opp.penul.pos_outside.bl,opp.penul.pos_sl.to.bl,penul.ball.bounce_sl.to.bl
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5718,4,35.515042,-0.021725,3.474766,6.797621,1.467570,14.1758,1.1448,13.6704,0.2330,32.313592,1.350533,2.449182,0.705435,12.5628,2.0724,0.445318,1,mens,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,0,1,0,1,0,1
371,4,33.382640,1.114202,2.540801,2.608708,2.311931,12.7396,4.6258,12.6358,2.3162,34.060788,0.760499,0.583291,3.856600,12.3544,5.1124,0.432434,1,mens,1,0,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,1,0,1,0,1
7948,23,22.316690,-0.254046,3.533166,9.435749,3.903728,15.3782,1.4336,13.6996,2.0908,39.806628,0.377732,1.115250,2.908892,13.8620,1.6564,0.397538,1,mens,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,1,0,1
9312,9,36.837309,0.766694,0.586885,3.342180,0.583745,13.7308,0.0312,13.1492,0.0188,25.018818,1.239033,3.256695,0.557554,14.2596,0.1606,0.671984,1,mens,0,1,0,0,1,0,1,0,0,0,1,1,0,0,0,1,0,1,0,1,0,1
7667,4,35.544208,0.116162,0.918725,5.499119,2.333456,15.1992,1.7596,14.1310,0.3150,36.790979,0.165731,1.431146,3.945317,11.3658,1.1082,0.340411,1,mens,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0,1,1


In [37]:
#split back to train and test
train_df_m = df_m.loc[men_traindex, :]
train_df_m['outcome'] = men_outcome
test_df_m = df_m.loc[men_testdex, :]


In [41]:
y = train_df_m["outcome"]
x = train_df_m.drop(["outcome","train","gender"], axis=1)
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=.30,random_state=1)


In [42]:
MLA = [    #Ensemble Methods
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),
    #GLM
    linear_model.LogisticRegressionCV(),
]
MLA_columns = []
MLA_compare = pd.DataFrame(columns = MLA_columns)
row_index = 0
for alg in MLA:
 
    predicted = alg.fit(x_train, y_train).predict(x_test)
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index,'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Train Accuracy'] = round(alg.score(x_train, y_train), 4)
    MLA_compare.loc[row_index, 'MLA Test Accuracy'] = round(alg.score(x_test, y_test), 4)
    clf_probs = alg.predict_proba(x_test)
    MLA_compare.loc[row_index, 'MLA Test Log loss']= log_loss(y_test, clf_probs)
    row_index+=1
    
MLA_compare.sort_values(by = ['MLA Test Log loss'], ascending = True, inplace = True)    
MLA_compare


,MLA Name,MLA Train Accuracy,MLA Test Accuracy,MLA Test Log loss
0,GradientBoostingClassifier,0.9340,0.8693,0.347479
2,LogisticRegressionCV,0.8620,0.8640,0.428554
1,RandomForestClassifier,0.9911,0.8407,0.741897


In [56]:

test_m=test_df_m.drop(["train","gender"], axis=1)
newdf=pd.DataFrame(ensemble.GradientBoostingClassifier().fit(x_train, y_train).predict_proba(test_m),index=test_m.index.values)

test_df_m_check=pd.concat([test_m,newdf],axis=1)

,rally,speed,net.clearance,distance.from.sideline,depth,player.distance.travelled,player.impact.depth,player.impact.distance.from.center,player.depth,player.distance.from.center,previous.speed,previous.net.clearance,previous.distance.from.sideline,previous.depth,opponent.depth,opponent.distance.from.center,previous.time.to.net,serve_2,hitpoint_F,hitpoint_U,hitpoint_V,outside.sideline_True,outside.baseline_True,same.side_True,previous.hitpoint_F,previous.hitpoint_U,previous.hitpoint_V,server.is.impact.player_True,pes.above.net_1,player.movement_fwd,player.movement_still,player.movement_very fwd,player.penul.pos_outside.bl,player.penul.pos_sl.to.bl,player.impact.pos_outside.bl,player.impact.pos_sl.to.bl,opp.penul.pos_outside.bl,opp.penul.pos_sl.to.bl,penul.ball.bounce_sl.to.bl,0,1,2
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
3818,8,28.137918,0.468679,1.966179,3.314536,2.681196,9.7524,1.8360,11.7466,0.0438,29.633220,1.174355,1.502747,3.463435,14.4494,3.6362,0.618826,0,1,0,0,1,0,0,1,0,0,0,1,1,0,0,0,1,0,1,1,0,1,0.873228,0.100468,0.026304
4314,5,38.966954,-0.019078,1.897606,7.236670,2.615940,10.9856,1.2494,13.0578,0.3472,25.249938,2.043395,3.241975,5.041623,16.7640,3.8780,0.859699,0,1,0,0,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,1,0,1,0.864238,0.020783,0.114979
8954,20,41.355933,0.311212,0.596189,2.599312,0.540255,13.4296,0.1386,13.4462,0.6786,32.749002,0.456805,4.071144,5.199691,13.1730,1.1952,0.472725,0,1,0,0,1,0,1,1,0,0,0,1,1,0,0,1,0,1,0,1,0,1,0.950121,0.035510,0.014369
4883,10,34.890887,0.400757,0.371935,3.156086,0.883450,12.7154,1.4566,13.4910,1.8796,36.088930,0.560687,3.026854,4.173466,13.9632,4.1212,0.468570,1,1,0,0,1,0,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,1,0.957063,0.032335,0.010602
6126,3,29.141742,1.620074,4.086164,0.523797,1.527865,11.2098,2.1880,11.2612,0.6610,34.483658,0.332496,2.600082,5.034880,12.2838,4.6912,0.416195,0,1,0,0,0,1,0,1,0,0,1,1,1,0,0,0,1,0,1,1,0,1,0.751147,0.240208,0.008645


In [61]:
# read data

train_df_w=pd.read_csv('dinesh/competitions/crowdanalytix/tennis/womens_train_file.csv',index_col='id')
test_df_w=pd.read_csv('dinesh/competitions/crowdanalytix/tennis/womens_test_file.csv',index_col='id')

#remove target variable
women_outcome=train_df_w['outcome'].copy()
train_df_w=train_df_w.drop('outcome',axis=1).copy()
test_df_w=test_df_w.drop('outcome',axis=1).copy()



# join both train and test dataset to prepare data
df_w=pd.concat([train_df_w,test_df_w])
women_traindex=train_df_w.index
women_testdex=test_df_w.index
del train_df_w
del test_df_w

#create variables
df_w['pes.above.net']=np.where(df_w['net.clearance']>0,1,0)

bins=[-20,-0.0000000000001,0,3.5,20]
labels=['back','still','fwd','very fwd']
df_w['player.movement']=pd.cut((df_w['player.depth']-df_w['player.impact.depth']), bins=bins, labels=labels)

bins1=[0,6.5,11.9,25]
labels1=['inside.sl','sl.to.bl','outside.bl']
df_w['player.penul.pos']=pd.cut((df_w['player.depth']), bins=bins1, labels=labels1)
df_w['player.impact.pos']=pd.cut((df_w['player.impact.depth']), bins=bins1, labels=labels1)
df_w['opp.penul.pos']=pd.cut((df_w['opponent.depth']), bins=bins1, labels=labels1)


bins2=[0,5.5,11.9]
labels2=['sl.to.bl','inside.sl']
df_w['penul.ball.bounce']=pd.cut((df_w['previous.depth']), bins=bins2, labels=labels2)


df_w = pd.get_dummies(df_w, columns=['serve','hitpoint','outside.sideline','outside.baseline','same.side','previous.hitpoint','server.is.impact.player','pes.above.net','player.movement','player.penul.pos','player.impact.pos','opp.penul.pos','penul.ball.bounce'],drop_first=True )


#split back to train and test
train_df_w = df_w.loc[women_traindex, :]
train_df_w['outcome'] = women_outcome
test_df_w = df_w.loc[women_testdex, :]

y1 = train_df_w["outcome"]
x1 = train_df_w.drop(["outcome","train","gender"], axis=1)
x1_train, x1_test, y1_train, y1_test = train_test_split(x,y,test_size=.30,random_state=1)


test_w=test_df_w.drop(["train","gender"], axis=1)
newdf_w=pd.DataFrame(ensemble.GradientBoostingClassifier().fit(x1_train, y1_train).predict_proba(test_w),index=test_w.index.values)
test_df_w_check=pd.concat([test_w,newdf_w],axis=1)


In [63]:
test_df_w_check.head()

,rally,speed,net.clearance,distance.from.sideline,depth,player.distance.travelled,player.impact.depth,player.impact.distance.from.center,player.depth,player.distance.from.center,previous.speed,previous.net.clearance,previous.distance.from.sideline,previous.depth,opponent.depth,opponent.distance.from.center,previous.time.to.net,serve_2,hitpoint_F,hitpoint_U,hitpoint_V,outside.sideline_True,outside.baseline_True,same.side_True,previous.hitpoint_F,previous.hitpoint_U,previous.hitpoint_V,server.is.impact.player_True,pes.above.net_1,player.movement_fwd,player.movement_still,player.movement_very fwd,player.penul.pos_outside.bl,player.penul.pos_sl.to.bl,player.impact.pos_outside.bl,player.impact.pos_sl.to.bl,opp.penul.pos_outside.bl,opp.penul.pos_sl.to.bl,penul.ball.bounce_sl.to.bl,0,1,2
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
4909,14,31.079000,-0.462328,1.629727,9.648428,4.851806,14.4142,2.5692,15.7868,2.0844,34.928103,0.828574,1.385091,0.819747,11.8308,0.1660,0.376395,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,1,0,0,1,1,0.165085,0.829620,0.005296
175,9,31.782142,1.284631,1.390645,0.831692,1.793047,12.5042,0.6324,13.0442,1.0774,30.040645,0.351958,1.739477,6.445967,13.9170,3.6252,0.567198,1,0,0,0,1,1,0,1,0,0,1,1,1,0,0,1,0,1,0,1,0,0,0.981674,0.016702,0.001623
1854,3,36.845491,-0.246532,1.838205,7.836234,3.103949,13.8058,3.0398,11.5878,0.8684,25.967708,1.644732,0.915300,2.575093,13.2422,1.9352,0.644381,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,1,1,0,1,0,1,0.949418,0.040159,0.010422
3033,6,40.569782,0.340860,0.405846,3.278491,2.295146,11.8888,2.7554,13.7710,1.4420,26.035498,1.093800,3.020381,4.831610,15.2506,0.6542,0.705400,0,1,0,0,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,1,0,1,0.039796,0.001630,0.958574
2958,5,17.863210,-0.006775,2.146275,7.561458,4.022607,6.8584,0.3232,10.8800,0.2332,25.090352,1.426527,3.027099,5.543246,14.3292,3.6614,0.728196,0,1,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,1,1,0,0,0.657497,0.112427,0.230076


In [66]:
test_df_w_check.to_csv('dinesh/competitions/crowdanalytix/tennis/womens_test_file_modeled.csv')
test_df_m_check.to_csv('dinesh/competitions/crowdanalytix/tennis/mens_test_file_modeled.csv')

In [67]:
#logisticregression
test_m1=test_df_m.drop(["train","gender"], axis=1)
newdf_m1=pd.DataFrame(linear_model.LogisticRegressionCV().fit(x_train, y_train).predict_proba(test_m1),index=test_m1.index.values)
test_df_m1_check=pd.concat([test_m1,newdf_m1],axis=1)



test_w1=test_df_w.drop(["train","gender"], axis=1)
newdf_w1=pd.DataFrame(linear_model.LogisticRegressionCV().fit(x1_train, y1_train).predict_proba(test_w1),index=test_w1.index.values)
test_df_w1_check=pd.concat([test_w1,newdf_w1],axis=1)

test_df_w1_check.to_csv('dinesh/competitions/crowdanalytix/tennis/modeled/womens_test_file_LR.csv')
test_df_m1_check.to_csv('dinesh/competitions/crowdanalytix/tennis/modeled/mens_test_file_LR.csv')

In [ ]:
#RandomForest
test_m1=test_df_m.drop(["train","gender"], axis=1)
newdf_m1=pd.DataFrame(linear_model.LogisticRegressionCV().fit(x_train, y_train).predict_proba(test_m1),index=test_m1.index.values)
test_df_m1_check=pd.concat([test_m1,newdf_m1],axis=1)



test_w1=test_df_w.drop(["train","gender"], axis=1)
newdf_w1=pd.DataFrame(linear_model.LogisticRegressionCV().fit(x1_train, y1_train).predict_proba(test_w1),index=test_w1.index.values)
test_df_w1_check=pd.concat([test_w1,newdf_w1],axis=1)

test_df_w1_check.to_csv('dinesh/competitions/crowdanalytix/tennis/modeled/womens_test_file_LR.csv')
test_df_m1_check.to_csv('dinesh/competitions/crowdanalytix/tennis/modeled/mens_test_file_LR.csv')

In [90]:
from sklearn.ensemble import ExtraTreesClassifier
# load the iris datasets
# fit an Extra Trees model to the data
model = ExtraTreesClassifier()
model.fit(x_train, y_train)
# display the relative importance of each attribute
name=pd.Series(x_train.columns.values)
importance=pd.Series(model.feature_importances_)
var_imp=pd.DataFrame({'name':name,'importance':importance})
var_imp.sort_values(['importance'],ascending=False)


,importance,name
21,0.108394,outside.sideline_True
28,0.108151,pes.above.net_1
22,0.081817,outside.baseline_True
12,0.045876,previous.distance.from.sideline
1,0.044849,speed
6,0.042542,player.impact.depth
2,0.041976,net.clearance
4,0.036110,depth
16,0.034883,previous.time.to.net
3,0.034794,distance.from.sideline


In [72]:
x_train.columns.values

array(['rally', 'speed', 'net.clearance', 'distance.from.sideline',
       'depth', 'player.distance.travelled', 'player.impact.depth',
       'player.impact.distance.from.center', 'player.depth',
       'player.distance.from.center', 'previous.speed',
       'previous.net.clearance', 'previous.distance.from.sideline',
       'previous.depth', 'opponent.depth', 'opponent.distance.from.center',
       'previous.time.to.net', 'serve_2', 'hitpoint_F', 'hitpoint_U',
       'hitpoint_V', 'outside.sideline_True', 'outside.baseline_True',
       'same.side_True', 'previous.hitpoint_F', 'previous.hitpoint_U',
       'previous.hitpoint_V', 'server.is.impact.player_True',
       'pes.above.net_1', 'player.movement_fwd', 'player.movement_still',
       'player.movement_very fwd', 'player.penul.pos_outside.bl',
       'player.penul.pos_sl.to.bl', 'player.impact.pos_outside.bl',
       'player.impact.pos_sl.to.bl', 'opp.penul.pos_outside.bl',
       'opp.penul.pos_sl.to.bl', 'penul.ball.bounce_sl.t